In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate as correlate
from stationprocessing import fir_filter_coefficients, channelize_ppf_contiguous_block

channels = 2048
taps = 16
sample_rate = 2.5e6

In [ ]:
#### Load the files here, use the [number:] if you already have a delay number to correct for
#### -n delay means [n:] for IQ0, n delay means [n:] for IQ1

IQ0 = np.fromfile("delay_test/GPS_clock_Airspy0", dtype="complex64")[1345:]
IQ1 = np.fromfile("delay_test/GPS_clock_Airspy1", dtype="complex64")

#IQ0 = np.fromfile("EXPERIMENT/airspy0_LNA", dtype="complex64")[26297:]
#IQ1 = np.fromfile("EXPERIMENT/airspy1_LNA", dtype="complex64")[:]

In [ ]:
#### Get the lengths of the files. These need not be equal, even after correcting for delay
#### since there also is a delay turning them off.
print("Original length:", len(IQ0))
print("Original length:", len(IQ1))

In [ ]:
#### resize arrays pre-calculation, to reduce needed cpu time
#### make sure the .copy() remains, since that will force a ram clean-up
#### 1s = 20MB/file = 2_500_000 samples/file, 1m = 1.2GB/file = 150_000_000 samples/file

resize_start = 8*60 #seconds
resize_end   = 10*60
IQ0 = IQ0[int(resize_start * sample_rate) : int(resize_end * sample_rate)].copy()
IQ1 = IQ1[int(resize_start * sample_rate) : int(resize_end * sample_rate)].copy()

print("Resized length:", len(IQ0))
print("Resized length:", len(IQ1))

In [ ]:
#### Plot magnitude of time-series data, to visually inspect start-up delay
plot_till = 80_000_000
step = 5_000
plt.plot(np.abs(IQ0[:plot_till][::step]), alpha=.6)
plt.plot(np.abs(IQ1[:plot_till][::step]), alpha=.6)
plt.xlabel("Number of steps")
plt.ylabel("Magnitude a.u.")
plt.title("Time-series magnitude as function of sample number")
plt.show()

In [ ]:
#### This code takes a while, only run on smaller subsets, or be prepared to wait a while
#### A very good measure however to test if your delay correction is correct.
#### The print should return 0, if the correction if proper, otherwise it prints the delay
#### It will onlt print the delay if the delay is constant, flat. If there is a non-constant delay, solve that first.
corr_abs = np.abs(correlate(IQ0, IQ1, mode='same', method='fft'))
plt.plot(corr_abs)
plt.xlabel("Sample number")
plt.ylabel("Correlation a.u.")
plt.title("Correlation as function of sample number")
plt.show()

print(len(IQ1)//2 - np.argmax(np.abs(corr_abs)))
%xdel corr_abs

In [ ]:
#### This code is significantly faster, and used to track dropped samples by testing
#### coherency as function of time. Ignore the first [start] terms, then correlate
#### a subarray of length [upto], and jump through the full array at steps [step].
#### A straight line is expected. If it's not at 0, there is a full-sample delay between the two.

#### If the line is not flat, you have big problems.
#### If the line is linearly increasing, your clocks are at different frequencies, thus the GPS clock is either not connected or fucked.
#### If the line is all over the place, you are dropping samples and your data is useless. Or your step is larger than the samplle delay.

step  = 100_000         # Make sure step is larger than initial delay, otherwise it cannot find coherency
upto  = 200_000         # good value is >1x step, to make sure you aren't skipping data.
start = 100_000         # skip the initial portion, which likely isnt good anyway
end   = 120_000_000     # make sure this is less than your total arraysize.

if end > len(IQ0): end = np.min((len(IQ0), len(IQ1)))

arr = []
for s in np.arange(start, end, step).astype('int'):
    c = correlate(IQ0[s:s+upto], IQ1[s:s+upto], mode='same')
    shift = upto // 2 - np.argmax(np.abs(c)) 
    arr += [shift]
plt.plot(arr)
plt.xlabel("Steps")
plt.ylabel("Sample offset")
plt.title("Sample offset as function of step")
plt.show()

print(np.mean(arr))

In [ ]:
#### This code is preparational code for the 2D crosscorrelation.
#### It makes sure the lengths are multiples of the channel count, and that the arrays are equal length.
#### changing length to a multiple of channels
len0 = (IQ0.shape[0]//channels) * channels
len1 = (IQ1.shape[0]//channels) * channels
shortest = np.min((len0, len1))

IQ0 = IQ0[:shortest]
IQ1 = IQ1[:shortest]

l = IQ0.shape[0] / channels
time_len = len(IQ0) / sample_rate
time_len1 = len(IQ1) / sample_rate
print("Size of array:", len(IQ0), "\nObservation length (s):", time_len)
print("Size of array:", len(IQ1), "\nObservation length (s):", time_len1)

In [ ]:
#### This cell reshapes the time-series data into channels.
#### The fit contains some coefficients for the PFB

IQ0 = IQ0.reshape((-1, channels))
IQ1 = IQ1.reshape((-1, channels))

fir = fir_filter_coefficients(channels, taps)

In [ ]:
#### This is the actual FFT.

IQ0 = channelize_ppf_contiguous_block(IQ0, fir)
IQ1 = channelize_ppf_contiguous_block(IQ1, fir)

In [ ]:
IQ0.shape

In [ ]:
#### This calculates the phase of the crosscorrelation
cross_ang = np.angle(IQ0 * np.conj(IQ1))

In [ ]:
#### This calculates the magnitude of the crosscorrelation
cross_abs = np.abs(IQ0 * np.conj(IQ1))

In [ ]:
#### Plotting the 2D result.
#### Freq is the frequency range in MHz, ext and asp are parameters for the plt.imshow plotting, change asp
#### if the plot is weirdly shaped.

#### Expected result is a phase plot that is stable in time, and linear as function of frequency. 
#### If this is not the case, and its chaos or more regions than [-1, 0, 1] in terms of phase as function of frequency,
#### then your delay is incorrect. Chaos means its fucked, more than 3 regions means there is likely a sub-sample delay
#### which the earlier sub-array correlator cannot see.

freq = [1420 - 1.25, 1420 + 1.25]
ext = [freq[0], freq[1], 0, time_len]
asp = IQ0.shape[0] / time_len * (freq[1] - freq[0]) / channels / 256

In [ ]:
#### PHASE
plt.imshow(cross_ang, aspect=asp, vmin=-np.pi, vmax=np.pi, cmap='seismic', extent=ext, origin='lower')
plt.colorbar()
plt.title("Phase")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Time (s)")
plt.tight_layout()
plt.show()

In [ ]:
#### MAGNITUDE
plt.imshow(cross_abs, aspect=asp, extent=ext, origin='lower', norm='log', vmin=5e-8, vmax=6e-7)
plt.colorbar()
plt.xlabel("Frequency (MHz)")
plt.ylabel("Time (s)")
plt.title("Magnitude")
plt.tight_layout()
plt.show()

In [ ]:
#### DEBUG plots, top one plots the magnitude as function of time, but summed over all frequencies,
#### bottom one is a histogram of the logarithm of all values in the full 2D magnitude diagram.
plt.plot(cross_abs.sum(axis=1))
plt.show()

plt.hist(np.log10(cross_abs.flatten()), bins=100)
plt.show()

In [ ]:
#### The sum of the phase over the time-axis can also be useful to see slight deviations.
#### It is plotted below.
#### If the phase plot is not a linear line going from -1 to 0 to 1, something is going wrong.
#### If the phase is chaos, its fucked, and if its smooth but not linear, then there is a sub-sample delay.

fig, (ax1, ax2) = plt.subplots(1, 2)
x = np.linspace(freq[0], freq[1], channels)
ax1.plot(x, cross_ang.mean(axis=0))
ax2.plot(x, cross_abs.mean(axis=0))
ax1.set_xlabel("Frequency (MHz)")
ax2.set_xlabel("Frequency (MHz)")
ax1.set_ylabel("Phase")
ax2.set_ylabel("Magnitude")
ax1.set_title("Phase")
ax2.set_title("Magnitude")
ax1.tick_params(rotation=30)
ax2.tick_params(rotation=30)
plt.suptitle("Mean phase and magnitude of the crosscorrelation\nof the shifted resistor observation")
plt.tight_layout()
plt.show()

In [ ]:
#### Code for sub-sample delay correction based on phase goes here.
#### (the code is mostly done, but its not here yet, ask Floris if you need it)